In [1]:
#Importacion de Librerias 

import pandas as pd 
import unicodedata
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import glob
import os
import re
import pandas as pd
from google.cloud import bigquery
import numpy as np
from google.oauth2 import service_account

In [2]:
#Funciones 
def limpiar_nombre_columna(col):
    col = ''.join(
        c for c in unicodedata.normalize('NFD', col)
        if unicodedata.category(c) != 'Mn'
    )
    col = re.sub(r'[^0-9a-zA-Z]+', '_', col)
    col = re.sub(r'_+', '_', col).strip('_')
    return col


def upload_pandas_dataframe_to_bigquery(dataframe, project_id, dataset_id, table_id, 
                                       credentials_path, if_exists='replace'):
    """
    Sube un DataFrame de pandas a una tabla de BigQuery.
    
    Args:
        dataframe: DataFrame de pandas con los datos a subir.
        project_id: ID del proyecto de Google Cloud.
        dataset_id: ID del dataset de BigQuery.
        table_id: ID de la tabla de BigQuery.
        if_exists: Acción a tomar si la tabla ya existe ('fail', 'replace', o 'append').
    """
    credentials = service_account.Credentials.from_service_account_file(
        credentials_path,
        scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )
    
    destination_table = f"{project_id}.{dataset_id}.{table_id}"
    dataframe.to_gbq(
        destination_table=destination_table,
        project_id=project_id,
        if_exists=if_exists,
        credentials=credentials
        
    )
    
    print(f"DataFrame subido exitosamente a {destination_table}")
    print(f"Total de filas subidas: {len(dataframe)}")

def limpiar_df_para_bigquery(df):
    """
    Limpia un DataFrame para hacerlo compatible con BigQuery:
    - Normaliza los nombres de columnas (reemplaza espacios y caracteres especiales con _)
    - Asegura que los nombres de columnas sean válidos para BigQuery
    
    Args:
        df: DataFrame de pandas a limpiar
        
    Returns:
        DataFrame limpio con nombres de columnas normalizados
    """
    df_limpio = df.copy()
    columnas_limpias = {}
    for col in df.columns:
        nuevo_nombre = col.strip()
        nuevo_nombre = re.sub(r'[^a-zA-Z0-9]', '_', nuevo_nombre)
        nuevo_nombre = re.sub(r'_+', '_', nuevo_nombre)
        nuevo_nombre = nuevo_nombre.strip('_')
        if nuevo_nombre and nuevo_nombre[0].isdigit():
            nuevo_nombre = 'col_' + nuevo_nombre
            
        if not nuevo_nombre:
            nuevo_nombre = f'columna_{df.columns.get_loc(col)}'
        columnas_limpias[col] = nuevo_nombre
    df_limpio.rename(columns=columnas_limpias, inplace=True)
    print("Nombres de columnas normalizados:")
    for original, nuevo in columnas_limpias.items():
        if original != nuevo:
            print(f"  • '{original}' → '{nuevo}'")
    df_limpio = df_limpio.astype(str)
    #drop duplicates
    df_limpio = df_limpio.drop_duplicates()

    #select columns 
    columnas_a_seleccionar =  [    'EAN_Empresa_Compradora',    'Nombre_Empresa_Compradora',    'Tipo_Orden',
    'N_mero_de_la_Orden',    'EAN_Entidad_a_Facturar',    'Nombre_Entidad_a_Facturar',    'EAN_Lugar_Entrega_Factura',
    'Nombre_Lugar_Entrega_Factura',    'F_Documento_O_C',    'F_M_nima_Entrega',    'F_M_xima_Entrega',
    'F_Solicitada_de_entrega',    'Acuerdo_Promocional',    'EAN_del_tem',    'C_digo_tem_proveedor',
    'Descripci_n_del_tem',    'EAN_Lugar_Entrega_tems',    'Nombre_Lugar_Entrega_tems',
    'Cantidad_Total',    'EAN_Punto_de_Venta',    'Nombre_Punto_de_Venta',
    'Cantidad_Pto_Vta',    'Precio_Bruto',    'Precio_Neto',
    'IVA',    'Valor_Impoconsumo',    'Cantidad_sub_art_culos'
]
    # Filtrar las columnas que existen en el DataFrame
    df_limpio = df_limpio[columnas_a_seleccionar]
    return df_limpio

In [3]:
# Ruta del archivo Excel
ruta_excel = "C:/Users/Lenovo/Downloads/scripts/OC - copia Daniel/main.xlsx"

# Leer Excel (primera hoja por defecto)
df = pd.read_excel(ruta_excel)

In [4]:
df_concatenado = df

In [5]:
df_concatenado.columns = [limpiar_nombre_columna(c) for c in df_concatenado.columns]

In [6]:
df_2 = df_concatenado.rename(columns={
                'EAN_Empresa_Compradora': 'EAN_Empresa_Compradora',
                'Razon_Social_Empresa_Compradora': 'Nombre_Empresa_Compradora',
                'Tipo_Orden_de_Compra': 'Tipo_Orden',
                'Numero_de_la_Orden_de_Compra': 'N_mero_de_la_Orden',
                'EAN_Entidad_a_Facturar': 'EAN_Entidad_a_Facturar',
                'Nombre_Entidad_a_Facturar': 'Nombre_Entidad_a_Facturar',
                'EAN_Lugar_Entrega_Factura': 'EAN_Lugar_Entrega_Factura',
                'Nombre_Entidad_a_Facturar': 'Nombre_Entidad_a_Facturar',
                'Nombre_Entidad_a_Facturar': 'Nombre_Entidad_a_Facturar',
                'F_Documento_O_C' : 'F_Documento_O_C',
                'F_Minima_Entrega' : 'F_M_nima_Entrega',
                'F_Maxima_Entrega' : 'F_M_xima_Entrega',
                'F_Solicitada_de_Entrega' : 'F_Solicitada_de_entrega',
                'Acuerdo_Promocional' : 'Acuerdo_Promocional',
                'EAN_del_Item' : 'EAN_del_tem',
                'Codigo_Item_Proveedor' : 'C_digo_tem_proveedor',
                'Descripcion_del_Item' : 'Descripci_n_del_tem',
                'EAN_Lugar_Entrega_Items':  'EAN_Lugar_Entrega_tems',
                'Nombre_Lugar_Entrega_Items': 'Nombre_Lugar_Entrega_tems',
                'Cantidad_Total' : 'Cantidad_Total',
                'EAN_Punto_de_Venta': 'EAN_Punto_de_Venta',
                'Nombre_Punto_de_Venta': 'Nombre_Punto_de_Venta',
                'Cantidad_Pto_Vta': 'Cantidad_Pto_Vta',
                'Precio_Bruto': 'Precio_Bruto',
                'Precio_Neto' : 'Precio_Neto',
                'IVA' : 'IVA',
                'Valor_Impoconsumo' : 'Valor_Impoconsumo',
                'Cantidad_sub_articulos' : 'Cantidad_sub_art_culos'
})


In [7]:
# Selección de columnas con los nombres normalizados
df_select = df_2[[
    'EAN_Empresa_Compradora',
    'Nombre_Empresa_Compradora',
    'Tipo_Orden',
    'N_mero_de_la_Orden',
    'EAN_Entidad_a_Facturar',
    'Nombre_Entidad_a_Facturar',
    'EAN_Lugar_Entrega_Factura',
    'Nombre_Lugar_Entrega_Factura',
    'F_Documento_O_C',
    'F_M_nima_Entrega',
    'F_M_xima_Entrega',
    'F_Solicitada_de_entrega',
    'Acuerdo_Promocional',
    'EAN_del_tem',
    'C_digo_tem_proveedor',
    'Descripci_n_del_tem',
    'EAN_Lugar_Entrega_tems',
    'Nombre_Lugar_Entrega_tems',
    'Cantidad_Total',
    'EAN_Punto_de_Venta',
    'Nombre_Punto_de_Venta',
    'Cantidad_Pto_Vta',
    'Precio_Bruto',
    'Precio_Neto',
    'IVA',
    'Valor_Impoconsumo',
    'Cantidad_sub_art_culos'
]]


In [8]:
df_select['F_Documento_O_C'] = pd.to_datetime(df_select['F_Documento_O_C'], format="%d/%m/%Y %H:%M").dt.strftime("%Y-%m-%d")


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19716\3709613972.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['F_Documento_O_C'] = pd.to_datetime(df_select['F_Documento_O_C'], format="%d/%m/%Y %H:%M").dt.strftime("%Y-%m-%d")


In [9]:
df_select['F_M_nima_Entrega'] = pd.to_datetime(df_select['F_M_nima_Entrega'], format="%d/%m/%Y %H:%M").dt.strftime("%Y-%m-%d")
df_select['F_M_xima_Entrega'] = pd.to_datetime(df_select['F_M_xima_Entrega'], format="%d/%m/%Y %H:%M").dt.strftime("%Y-%m-%d")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19716\101682642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['F_M_nima_Entrega'] = pd.to_datetime(df_select['F_M_nima_Entrega'], format="%d/%m/%Y %H:%M").dt.strftime("%Y-%m-%d")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19716\101682642.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['F_M_xima_Entrega'] = pd.to_datetime(df_select['F_M_xima_Entrega'], format="%d/%m/%Y %H:%M").dt.strftime("%Y-%m-%d")


In [10]:
df_select.head()

,EAN_Empresa_Compradora,Nombre_Empresa_Compradora,Tipo_Orden,N_mero_de_la_Orden,EAN_Entidad_a_Facturar,Nombre_Entidad_a_Facturar,EAN_Lugar_Entrega_Factura,Nombre_Lugar_Entrega_Factura,F_Documento_O_C,F_M_nima_Entrega,...,Nombre_Lugar_Entrega_tems,Cantidad_Total,EAN_Punto_de_Venta,Nombre_Punto_de_Venta,Cantidad_Pto_Vta,Precio_Bruto,Precio_Neto,IVA,Valor_Impoconsumo,Cantidad_sub_art_culos
0,7700304892105,D1 Sas,Orden de Compra,4503206402,7700304892105,D1 Sas,7700304046973,6A220900 CD CUCUTA,2025-10-23,2025-10-30,...,6A220900 CD CUCUTA,1440.0,7700304046973,6A220900 CD CUCUTA,NaN,6050.0,6050.0,5.0,1210.0,0.0
1,7700304892105,D1 Sas,Orden de Compra,4503206402,7700304892105,D1 Sas,7700304046973,6A220900 CD CUCUTA,2025-10-23,2025-10-30,...,6A220900 CD CUCUTA,4320.0,7700304046973,6A220900 CD CUCUTA,NaN,2338.0,2338.0,5.0,467.6,0.0
2,7700304892105,D1 Sas,Orden de Compra,4503206401,7700304892105,D1 Sas,7700304846535,6A100900 CD Bucaramanga,2025-10-23,2025-10-30,...,6A100900 CD Bucaramanga,4320.0,7700304846535,6A100900 CD Bucaramanga,NaN,2338.0,2338.0,5.0,467.6,0.0
3,7701001000008,ALMACENES EXITO S.A,Orden de Compra de Cross- Docking Predistribuido,20666071,7701001000008,ALMACENES EXITO S.A,7701001000305,3 - CEDI VEGAS,2025-10-23,2025-10-29,...,3 - CEDI VEGAS,150.0,7701001033808,338 - CARULLA OVIEDO,30.0,2837.1,2837.1,NaN,NaN,0.0
4,7701001000008,ALMACENES EXITO S.A,Orden de Compra de Cross- Docking Predistribuido,20666071,7701001000008,ALMACENES EXITO S.A,7701001000305,3 - CEDI VEGAS,2025-10-23,2025-10-29,...,3 - CEDI VEGAS,NaN,7701001552002,552 - CARULLA VISITACION,30.0,2837.1,2837.1,NaN,NaN,0.0


In [11]:


if __name__ == "__main__":
    
    credentials_path = "credentials/croc-454221-e1a3c2e02181.json"
    
    df = limpiar_df_para_bigquery(df_select)
    project_id = "croc-454221"
    dataset_id = "db_sales"
    table_id = "tbl_brz_sales_orders_canal_moderno_temp"
    
    upload_pandas_dataframe_to_bigquery(
        df, 
        project_id, 
        dataset_id, 
        table_id, 
        credentials_path,
        if_exists='append'
    )

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19716\374761916.py:30: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  dataframe.to_gbq(


Nombres de columnas normalizados:


100%|██████████| 1/1 [00:00<00:00, 12945.38it/s]

DataFrame subido exitosamente a croc-454221.db_sales.tbl_brz_sales_orders_canal_moderno_temp
Total de filas subidas: 273


In [12]:
df_select.head(2)

,EAN_Empresa_Compradora,Nombre_Empresa_Compradora,Tipo_Orden,N_mero_de_la_Orden,EAN_Entidad_a_Facturar,Nombre_Entidad_a_Facturar,EAN_Lugar_Entrega_Factura,Nombre_Lugar_Entrega_Factura,F_Documento_O_C,F_M_nima_Entrega,...,Nombre_Lugar_Entrega_tems,Cantidad_Total,EAN_Punto_de_Venta,Nombre_Punto_de_Venta,Cantidad_Pto_Vta,Precio_Bruto,Precio_Neto,IVA,Valor_Impoconsumo,Cantidad_sub_art_culos
0,7700304892105,D1 Sas,Orden de Compra,4503206402,7700304892105,D1 Sas,7700304046973,6A220900 CD CUCUTA,2025-10-23,2025-10-30,...,6A220900 CD CUCUTA,1440.0,7700304046973,6A220900 CD CUCUTA,NaN,6050.0,6050.0,5.0,1210.0,0.0
1,7700304892105,D1 Sas,Orden de Compra,4503206402,7700304892105,D1 Sas,7700304046973,6A220900 CD CUCUTA,2025-10-23,2025-10-30,...,6A220900 CD CUCUTA,4320.0,7700304046973,6A220900 CD CUCUTA,NaN,2338.0,2338.0,5.0,467.6,0.0
